In [1]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 1

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
#tf.random.set_seed(seed_value)
# for later versions: 
tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
#session_conf = tf.ConfigProto()
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)
# for later versions:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import SGD
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.applications.vgg16 import VGG16
from keras.layers import LSTM
#import numpy as np
import glob
from imageio import imread
from skimage.transform import resize as imresize
from keras.layers import LeakyReLU
import pandas as pd
import numpy as np
seq = 120

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [77]:
train = pd.read_csv('train_zero_padded.csv')
val = pd.read_csv('validation_zero_padded_6.csv')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2053) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,2054,2060,2061,2065,2066,2067,2068) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [78]:
val.drop('labels',axis=1,inplace=True)
#val.drop('t',axis=1,inplace=True)


In [79]:
train.drop('Unnamed: 0',axis=1,inplace=True)
train.drop('tags',axis=1,inplace=True)
val.drop('Unnamed: 0',axis=1,inplace=True)
#val.drop('Unnamed: 0.1',axis=1,inplace=True)
val.drop('tags',axis=1,inplace=True)

In [80]:
#train.drop('Unnamed: 0.1',axis=1,inplace=True)

KeyError: "['Unnamed: 0.1'] not found in axis"

In [81]:
train.drop('labels',axis=1,inplace=True)

In [87]:
s1 = set(train.columns)
s2 = set(val.columns)

In [88]:
s2 - s2.intersection(s1)

set()

In [84]:
val.drop(['From (min.sec)',
 'Label',
 'Procedure or manipulation (Name)',
 'To (min.sec)',
 'Total Duration of video file (min.sec)',
 'UHID',
 'Video (Title)',
 't',
 'time','title_y'],axis=1,inplace=True)

In [85]:
val.rename(columns={'title_x':'title'},inplace=True)

In [89]:
df = pd.concat([train,val])

In [90]:
ids = df.uhid.unique()

In [91]:
model = Sequential()
#model.add(LSTM(256,dropout=0.2,input_shape=(None,2048)))
model.add(LSTM(512,return_sequences=True,activation='tanh',input_shape=(None,2048)))
#model.add(activation='tanh')
model.add(LSTM(256,return_sequences=True))
model.add(LSTM(128,return_sequences=False))
#model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
es =   EarlyStopping(monitor='val_loss', patience=8, verbose=0)

In [92]:
X = pd.DataFrame()

In [93]:
for i in ids:
    patient = df[df['uhid']==i]
    total_frames = patient['Total_frames'].unique()
    for j in total_frames:
        x = patient[patient['Total_frames']==j]
        x = x.sort_values(by=['series','frame_number'])             
        x['diaper_change'].fillna(method='ffill',inplace=True)
        x['feeding'].fillna(method='ffill',inplace=True)
        x['patting'].fillna(method='ffill',inplace=True)
        X = X.append(x,ignore_index=True)

In [94]:
X.drop('title',axis=1,inplace=True)
#X.drop('uhid',axis=1,inplace=True)
X.drop('Total_frames',axis=1,inplace=True)
X.drop('frame_number',axis=1,inplace=True)
X.drop('series',axis=1,inplace=True)
#X.drop('labels',axis=1,inplace=True)

In [95]:
#X.drop('labels',axis=1,inplace=True)

In [96]:
#X.drop('tags',axis=1,inplace=True)

In [97]:
len(train)/120

319.0

In [98]:
pred = []
test = []

In [99]:
ids = df.uhid.unique()

In [100]:
for i in ids:
    #try:
    print(i)
    X_val = X[X['uhid']==i]
    X_train = X[X['uhid']!=i]
    X_val.drop('uhid',axis=1,inplace=True)
    X_train.drop('uhid',axis=1,inplace=True)
    train_labels1 = X_train[['diaper_change','feeding','patting']]
    validation_labels1 = X_val[['diaper_change','feeding','patting']]
    train_labels = []
    for i in range(0,len(train_labels1),seq):
    #print(i)
        train_labels1 = np.array(train_labels1)
        y1 = train_labels1[i:i+seq]
        train_labels.append(y1[-4])

    validation_labels = []
    for i in range(0,len(validation_labels1),seq):
    #print(i)
        validation_labels1 = np.array(validation_labels1)
        y2 = validation_labels1[i:i+seq]
        validation_labels.append(y2[-4])
    X_train.drop(['diaper_change','feeding','patting'],axis=1,inplace=True)
    X_val.drop(['diaper_change','feeding','patting'],axis=1,inplace=True)
    
    X_train = np.array(X_train)
    if len(X_train)%120 == 0:
        
        X_train = X_train.reshape(-1,seq,X_train.shape[1])
        
    else:
        n = int(len(X_train)/120) + 1
        m = n*120 - len(X_train)
        z = np.zeros((m,2048))
        X_train = np.vstack((X_train,z))
        X_train = X_train.reshape(-1,seq,X_train.shape[1])
        
    X_val = np.array(X_val)
    if len(X_val)%120 == 0:
        
        X_val = X_val.reshape(-1,seq,X_val.shape[1])
        
    else:
        n = int(len(X_val)/120) + 1
        m = n*120 - len(X_val)
        z = np.zeros((m,2048))
        X_val = np.vstack((X_val,z))
        
        X_val = X_val.reshape(-1,seq,X_val.shape[1]) 
        
    
   
    train_labels = np.array(train_labels)
    validation_labels = np.array(validation_labels)
    model.fit(X_train,train_labels,batch_size=120,epochs=40,validation_split=0.2,callbacks=[es])
    answer = model.predict(X_val)
    pred.append(answer)
    test.append(validation_labels)
    #except:
    #    continue


200101135


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Train on 309 samples, validate on 78 samples
Epoch 1/40
309/309 [==============================] - 6s 19ms/step - loss: 1.1052 - accuracy: 0.4498 - val_loss: 0.8888 - val_accuracy: 0.5385
Epoch 2/40
309/309 [==============================] - 5s 17ms/step - loss: 0.7326 - accuracy: 0.7540 - val_loss: 0.4045 - val_accuracy: 0.8718
Epoch 3/40
309/309 [==============================] - 5s 16ms/step - loss: 0.4148 - accuracy: 0.8964 - val_loss: 0.2153 - val_accuracy: 0.9487
Epoch 4/40
309/309 [==============================] - 5s 17ms/step - loss: 0.3048 - accuracy: 0.9191 - val_loss: 0.2997 - val_accuracy: 0.9231
Epoch 5/40
309/309 [==============================] - 5s 17ms/step - loss: 0.2226 - accuracy: 0.9353 - val_loss: 0.3497 - val_accuracy: 0.9103
Epoch 6/40
309/309 [==============================] - 5s 16ms/step - loss: 0.1664 - accuracy: 0.9385 - val_loss: 0.3799 - val_accuracy: 0.8974
Epoch 7/40
309/309 [==============================] - 5s 16ms/step - loss: 0.1227 - accuracy: 0.9

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Train on 180 samples, validate on 46 samples
Epoch 1/40
180/180 [==============================] - 3s 17ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 1.2564 - val_accuracy: 0.6739
Epoch 2/40
180/180 [==============================] - 3s 17ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 1.2361 - val_accuracy: 0.6522
Epoch 3/40
180/180 [==============================] - 3s 18ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 1.2628 - val_accuracy: 0.6522
Epoch 4/40
180/180 [==============================] - 3s 17ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 1.3208 - val_accuracy: 0.6522
Epoch 5/40
180/180 [==============================] - 3s 17ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 1.3870 - val_accuracy: 0.6522
Epoch 6/40
180/180 [==============================] - 3s 17ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 1.4488 - val_accuracy: 0.6522
Epoch 7/40
180/180 [==============================] - 3s 17ms/step - loss: 0.0014 - accuracy: 1.0

Epoch 11/40
312/312 [==============================] - 5s 16ms/step - loss: 4.7240e-04 - accuracy: 1.0000 - val_loss: 0.5232 - val_accuracy: 0.9367
Epoch 12/40
312/312 [==============================] - 5s 16ms/step - loss: 4.1795e-04 - accuracy: 1.0000 - val_loss: 0.5285 - val_accuracy: 0.9367
Epoch 13/40
312/312 [==============================] - 5s 16ms/step - loss: 3.8819e-04 - accuracy: 1.0000 - val_loss: 0.5333 - val_accuracy: 0.9367
Epoch 14/40
312/312 [==============================] - 5s 16ms/step - loss: 3.6811e-04 - accuracy: 1.0000 - val_loss: 0.5373 - val_accuracy: 0.9367
Epoch 15/40
312/312 [==============================] - 5s 16ms/step - loss: 3.5208e-04 - accuracy: 1.0000 - val_loss: 0.5408 - val_accuracy: 0.9367
Epoch 16/40
312/312 [==============================] - 5s 16ms/step - loss: 3.3935e-04 - accuracy: 1.0000 - val_loss: 0.5439 - val_accuracy: 0.9367
Epoch 17/40
312/312 [==============================] - 5s 16ms/step - loss: 3.2838e-04 - accuracy: 1.0000 - val_

285/285 [==============================] - 5s 17ms/step - loss: 1.3957e-04 - accuracy: 1.0000 - val_loss: 0.6491 - val_accuracy: 0.8889
Epoch 3/40
285/285 [==============================] - 5s 17ms/step - loss: 1.3776e-04 - accuracy: 1.0000 - val_loss: 0.6502 - val_accuracy: 0.8889
Epoch 4/40
285/285 [==============================] - 5s 17ms/step - loss: 1.3614e-04 - accuracy: 1.0000 - val_loss: 0.6513 - val_accuracy: 0.8889
Epoch 5/40
285/285 [==============================] - 5s 17ms/step - loss: 1.3446e-04 - accuracy: 1.0000 - val_loss: 0.6525 - val_accuracy: 0.8889
Epoch 6/40
285/285 [==============================] - 5s 17ms/step - loss: 1.3281e-04 - accuracy: 1.0000 - val_loss: 0.6538 - val_accuracy: 0.8889
Epoch 7/40
285/285 [==============================] - 5s 17ms/step - loss: 1.3117e-04 - accuracy: 1.0000 - val_loss: 0.6551 - val_accuracy: 0.8889
Epoch 8/40
285/285 [==============================] - 5s 17ms/step - loss: 1.2968e-04 - accuracy: 1.0000 - val_loss: 0.6563 - val

In [70]:
X_train

array([[[ 8.09753800e-02,  7.51085040e-01, -4.37508850e-01, ...,
         -7.76825550e-01,  5.72837230e-01, -2.59454160e-01],
        [ 2.51555610e-02, -6.60803800e-01, -1.11122030e-01, ...,
          5.31326530e-01, -1.64661350e-01,  1.06762810e+00],
        [-9.07105700e-01,  5.16185050e-01, -2.01945190e-01, ...,
          3.18011460e-02, -5.39899950e-01,  9.36408160e-01],
        ...,
        [-3.47070540e-01,  7.55162060e-01, -1.28870210e+00, ...,
         -8.85967730e-01,  7.35638740e-01,  4.63672400e-01],
        [-3.59686670e-01,  7.40027400e-01, -3.70151500e-01, ...,
         -1.99373400e-03,  1.95032720e-01, -5.12382030e-01],
        [ 7.31410300e-01, -2.05360920e-01, -1.11234710e+00, ...,
          3.00454380e-01,  6.67156600e-01, -1.92144980e-01]],

       [[-4.68390350e-01,  1.49111380e-01, -8.57686640e-01, ...,
          7.74056100e-01,  2.14054330e-01,  1.17967450e+00],
        [ 6.91929740e-03,  7.66926350e-01, -8.85070600e-01, ...,
          4.10388500e-01, -5.97938950e

In [115]:
y_tp = []
y_pred = []

In [102]:
for i in range(len(test)):
    p = pred[i]
    t = test[i]
    
    for j in t:
        print(j)
        y_tp.append(np.argmax(j))
    for k in p:
        print(k)
        y_pred.append(np.argmax(k))
    

[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[0.96479404 0.03280864 0.00239733]
[0.00180136 0.97734195 0.02085672]
[5.977019e-04 8.088140e-04 9.985935e-01]
[5.3162180e-04 8.0230029e-04 9.9866605e-01]
[5.1527814e-04 7.6086610e-04 9.9872380e-01]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 1. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[0. 0. 1.]
[1. 0. 0.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[0. 0. 1.]
[1. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 0. 1.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.

In [73]:
from sklearn.metrics import classification_report,confusion_matrix

In [119]:
print(classification_report(y_tp,y_pred))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95        63
           1       1.00      0.89      0.94       108
           2       0.98      1.00      0.99       221

    accuracy                           0.97       392
   macro avg       0.96      0.96      0.96       392
weighted avg       0.97      0.97      0.97       392



In [120]:
print(confusion_matrix(y_tp,y_pred))

[[ 63   0   0]
 [  7  96   5]
 [  0   0 221]]
